## 4.10 Coding Etiquette & Excel Reporting Part 1
### This script will include
#### 00. Importing libraries and data
#### 01. Considering security implications and any PII data
#### 02. Creating a regional segmentation of the data
#### 03. Create an exclusion flag for low activity customers

### 00. Importing libraries and data

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Create a folderpath shortcut

path = r'C:\Users\Victoria\Desktop\Data Analytics Career Foundry\4.0 Python Fundamentals for Data Analysts\Instacart Basket Analysis'

In [5]:
# Import 'orders_products_customers_merged.pkl' as ic_data

ic_data = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_merged.pkl'))

In [6]:
# Check import

ic_data.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [7]:
ic_data.shape

(32404859, 33)

### 01. Considering security implications and any PII data

The 'first_name' and 'last_name' columns contain personally identifiable information and need to be removed.

In [8]:
# Drop the 'first_name' and 'last_name' columns due to PII data

ic_data.drop(['first_name', 'last_name'], axis = 1, inplace = True)

In [9]:
# Confirm column removals

ic_data.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_order_days,order_frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


### 02. Creating a regional segmentation of the data

Determine whether there’s a difference in spending habits between the different U.S. regions.

In [10]:
# Categorize regions based on regions defined from exercise source. Found in final report.

northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [11]:
# Assign states to corresponding region

ic_data.loc[ic_data['state'].isin(northeast), 'region'] = 'Northeast'
ic_data.loc[ic_data['state'].isin(midwest), 'region'] = 'Midwest'
ic_data.loc[ic_data['state'].isin(south), 'region'] = 'South'
ic_data.loc[ic_data['state'].isin(west), 'region'] = 'West'

In [12]:
# Check no values for 'state' were excluded

ic_data['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [13]:
# Create a crosstab of spending habits between different regions in the US with 'regions' and 'spending flag'

regional_spending = pd.crosstab(ic_data['region'],ic_data['spending_flag'], dropna = False)

In [14]:
# Examine results

regional_spending

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [15]:
# I'm going to create a percentage calculation because the format of the data as it is isn't very helpful

regional_spending['total_spending'] = regional_spending.sum(axis=1)
regional_spending['%_high_spender'] = (regional_spending['High spender'] / regional_spending['total_spending']) * 100
regional_spending['%_low_spender'] = (regional_spending['Low spender'] / regional_spending['total_spending']) * 100

In [16]:
regional_spending

spending_flag,High spender,Low spender,total_spending,%_high_spender,%_low_spender
region,,,,,
Midwest,155975,7441350,7597325,2.053025,97.946975
Northeast,108225,5614511,5722736,1.891141,98.108859
South,209691,10582194,10791885,1.943043,98.056957
West,160354,8132559,8292913,1.933627,98.066373


In [17]:
# Copy crosstab to Excel workbook

regional_spending.to_clipboard()

##### Explanation

Based on these results, spending habits appear to be similar across all regions. Low spenders outnumber high spender in all regions. The Midwest has the highest proportion of high spenders (2.05%) and the Northeast has the highest proportion of low spenders (98.12%). However, all of these differences in spending are very slight.

### 03. Create an exclusion flag for low activity customers

The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [18]:
# Create a statement for 'High activity customer'

ic_data.loc[ic_data['max_order'] >=5,'customer_activity'] = 'High-activity customer'

In [19]:
# Create a statement for 'Low-activity customer'

ic_data.loc[ic_data['max_order'] <5, 'customer_activity'] = 'Low-activity customer'

In [20]:
# Check results to ensure no values were excluded

ic_data['customer_activity'].value_counts(dropna = False)

High-activity customer    30964564
Low-activity customer      1440295
Name: customer_activity, dtype: int64

In [21]:
# Create a subset of low-activity customers

#df_low = ic_data[ic_data['customer_activity'] == 'Low-activity customer']

In [22]:
# Create a subset of high-activity customers

#df_high = ic_data[ic_data['customer_activity'] == 'High-activity customer']

In [23]:
# Export low-activity customer data, incase in the future IC wants to look at this data

#df_low.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ic_data_low_activity_customers'))

In [24]:
# Export high-activity customer data

#df_high.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ic_data_high_activity_customers'))

In [25]:
# Export ic_data with changes for continuation in part 2

#ic_data.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ic_data_4_10_p1.pkl'))

### Remaining assignment continued in 4.10 IC Coding Etiquete & Excel Reporting - Part 2